# 第五章 不确定度计算

**不确定度计算部分**能够根据实验数据、实验仪器和测量公式，计算**单个独立测量**的不确定度，乃至**通过测量公式得到的最终测量值**的不确定度。<br>analyticlab的不确定度计算由`Measure`（测量）和`Uncertainty`（不确定度）类实现，每个独立测量所使用的仪器用`Ins`（仪器）类表示。`Measure`为单个测量，各个测量Measure之间通过数学关系式进行运算，得到合成不确定度Uncertainty。单个测量可以直接通过类方法实现生成并展示其不确定度的计算过程；对于Uncertainty，可以通过`latexoutput`函数模块的`dispUnc`函数实现生成并展示合成不确定度（如果有需要，还可以是扩展不确定度）的计算过程。<br>`std`、`ACategory`、`BCategory`函数模块是实现不确定度计算的**内部逻辑模块**。这些函数模块在实际应用过程中通常不会被直接调用，感兴趣的用户可以了解一下这些函数模块。

#### 本章涉及的类与模块：
* Ins类 - analyticlab.uncertainty.ins
* Measure类 - analyticlab.uncertainty.measure
* Uncertainty类 - analyticlab.uncertainty.unc
* (部分了解)amath函数模块 - analyticlab.amath
* (部分了解)latexoutput函数模块 - analyticlab.latexoutput
* \* std函数模块 - analyticlab.uncertainty.std
* \* ACategory函数模块 - analyticlab.uncertainty.ACategory
* \* BCategory函数模块 - analyticlab.uncertainty.BCategory

## 1.Ins类：描述测量仪器

`Ins`类用于描述一个测量仪器，包括仪器不确定度的半宽度、分布类型和测量值单位。`ins`模块自身内置了一些常用的测量仪器，可以通过`dir(ins)`查看是否有现成的仪器对象可以直接使用。如果没有，需要通过`Ins`类创建一个仪器对象。

### 1.1 ins函数模块

#### 1.1.1 导入ins函数模块

当需要使用现成的仪器对象时，可以在ins函数模块中直接引用。ins函数模块位于`analyticlab.uncertainty`包内，通过以下指令实现导入：

In [1]:
from analyticlab.uncertainty import ins

#### 1.1.2 查找现成的仪器对象

In [2]:
dir(ins)

['Ins',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 '一级千分尺_毫米_3',
 '刻度尺_毫米_1',
 '显微镜螺旋测微器_毫米_3',
 '游标卡尺_厘米_3',
 '游标卡尺_毫米_2',
 '游标卡尺_米_5',
 '米尺_厘米_1',
 '米尺_厘米_2',
 '米尺_米_4']

在通过`dir`查找到的对象中，以`“仪器名称_测量值单位_测量值为小数点后几位”`为名称的对象即为现成的仪器对象（分布类型均为矩形分布）。当确保现成仪器对象的单位、测量精度、分布类型均与实际使用的仪器一致时，可以直接引用该仪器。

### 1.2 Ins类

#### 1.2.1 导入Ins类

Ins类位于`analyticlab.uncertainty.ins`模块内，通过以下指令实现导入：

In [3]:
from analyticlab.uncertainty.ins import Ins

#### 1.2.2 创建一个测量仪器对象

Ins类的构造方法如下：

`Ins(halfWidth, distribution=Ins.rectangle, unit=None, *param)`

其中参数`halfWidth`和`distribution`用于描述仪器不确定度：`halfWidth`为半宽度；`distribution`为分布类型，默认为矩形分布。`distribution`只能在下表值中选取。`unit`是测量值单位，可以不给出。`*param`为附加参数，是否需要给出与分布类型有关，当分布类型为梯形分布时，需要通过`*param`给出$\beta$值。

* `Ins.norm` - 正态分布
* `Ins.rectangle` - 矩形分布
* `Ins.triangle` - 三角分布
* `Ins.arcsin` - 反正弦分布
* `Ins.twopoints` - 两点分布
* `Ins.trapezoid` - 梯形分布，需要给出$\beta$值

下面举例说明如何创建一个测量仪器对象：

In [4]:
一级千分尺_毫米_3 = Ins('0.004', unit='mm')

## 2.Measure类：描述每个独立测量

`Mesure`类用于描述单个独立测量，包括**测量数据**（可以是单个数据、单个样本或多个样本）、**测量仪器**、**测量值的符号和单位**。测量数据必须给出，当测量数据超过一个时，将会作为计算A类不确定度的依据；测量仪器可选择是否给出，给出时，将会作为计算B类不确定度的依据。

### 2.1 导入Measure类

Measure类位于`analyticlab.uncertainty.measure`模块内，通过以下指令实现导入：

In [5]:
from analyticlab.uncertainty.measure import Measure

### 2.2 控制以何种方法计算A类不确定度

在Measure类中，静态类属性`Measure.AMethod`决定使用何种方法计算A类不确定度。`Measure.AMethod`只能是以下值中的一个：
* `'auto'`（默认值） - 根据样本数量的大小，决定使用那种方法，即样本数量最大的组为9以上时，使用Bessel法；否则用极差法
* `'Bessel'` - 使用贝塞尔公式法
* `'Range'`：使用极差法
* `'CollegePhysics'`：使用大学物理实验中的不确定度公式

当需要改变Measure.AMethod时，建议在创建测量之前，先改变Measure.AMethod的值，且只改变一次，中途不要再继续修改。通常情况下不需要修改Measure.AMethod，直接使用`'auto'`即可。

### 2.3 创建一个测量

Measure类的构造方法如下：

`Measure(data, instrument=None, sym=None, unit=None, description=None)`

参数`data`为测量数据，可以以**字符串形式**给出单个数值、一组数值作为测量数据；也可以给出Num、LSym（含数值）格式的单个数值，或者NumItem、LSymItem格式的单个样本；也可以以list给出，包括Num列表、LSym列表组成的单个样本，或者NumItem列表、LSymItem列表、每组数值分别用一个字符串表示出来的列表。当`data`为大于一个数值（包括单个样本和多个样本）时，会计算该测量的A类不确定度。例如：

* `'3.010'` - 单个数值
* `'3.007 3.010 3.008'` - 单个样本
* `['3.007 3.010 3.008', '3.005 3.009 3.007', '2.996 3.001 2.999', '3.001 2.998 2.997', '3.006 3.009 3.011']` - 多个样本

参数`instrument`为测量仪器。当给出测量仪器时，会计算该测量的B类不确定度。

参数`sym`和`unit`仅在需要**生成并展示不确定度的计算过程**时，才有必要考虑是否给出：
* `sym`为测量值的符号。当`data`以有sym的数据类型（比如LSym、LSymItem、NumItem）给出时，会自动取出data中的sym作为测量值的符号，因此不需要额外附加参数sym；附加参数sym时，会优先采用附加参数中的sym。而当`data`为其它数据类型时，必须给出参数sym。
* `unit`为测量值的单位。当测量仪器`instrument`给出时，使用测量仪器的单位；附加参数unit时，会优先采用附加参数中的unit。而当`instrument`未给出时，必须给出参数unit。

总之，在需要计算过程时，`sym`和`unit`是否一定需要给出，由前面的参数是否已包含sym、unit而定。

参数`description`是对测量的语言描述，比如`“小球直径$d$”`。当需要通过dispUnc函数展示合成不确定度或扩展不确定度的计算过程时，需要给出`description`，否则无需给出。

**下面举例说明测量的创建：**

* 以字符串形式给出测量数据，给出了测量仪器：

In [6]:
d1 = Measure('0.5069 0.5081 0.5066 0.5075 0.5073', ins.米尺_米_4)  #因为不需要计算过程，没有给出sym、unit

* 以字符串形式给出测量数据，未给出测量仪器，给出了符号和单位：

In [7]:
d1 = Measure('0.5069 0.5081 0.5066 0.5075 0.5073', sym='d1', unit='m')

* 由NumItem导入测量数据，给出了测量仪器，符号由NumItem导入，单位由测量仪器导入：

In [8]:
from analyticlab.numitem import NumItem
d1_data = NumItem('0.5069 0.5081 0.5066 0.5075 0.5073', sym='d_1')
d1 = Measure(d1_data, ins.米尺_米_4) #sym、unit都没必要给出

### 2.4 获得测量的标准不确定度

通过调用`unc`方法，可以获得独立测量的不确定度。unc方法如下：

`def unc(process=False, needValue=False)`

要想获得独立测量的不确定度，则无需传入任何参数，此时返回值为Num类型不确定度数值：

In [9]:
d1.unc()

4.1e-04

#### 2.4.1 展示计算过程

通过附加参数`process`和`needValue`，可以控制是否生成计算过程，以及生成计算过程的同时是否给出不确定度数值：

In [10]:
d1.unc(process=True).show()

<IPython.core.display.Math object>

### 2.5 获得测量的扩展不确定度

默认情况下，`unc`方法返回的是P=0.6826时的标准不确定度。要获得扩展不确定度，则需要调用`setK(K)`方法，给出**K值**。给定K值后，再调用`unc`方法，得到的就是扩展不确定度。要设定的K值只能是下列值：
* `0.67` - 对应P=0.50
* `1.645` - 对应P=0.90
* `1.960` - 对于P=0.95
* `2` - 对应P=0.9545
* `3` - 对应P=0.9973

<i>注意：如果要得到的最终测量结果是通过各独立测量值通过测量公式得到的，那么不要设置各个独立测量值的K值，只需要设置最终测量结果的K值。（第3节会提及Uncertainty的K值设置）</i>

## 3.Uncertainty类：获得合成不确定度

`Uncertainty`类用于描述**合成不确定度**，它不能被直接创建，只能由各个独立测量通过数学关系式表达出来。

### 3.1 控制是否生成计算表达式

在Uncertainty类中，静态类属性`Uncertainty.process`决定各测量参与运算时，是否生成不确定度的计算表达式。默认`Uncertainty.process=False`。当需要使用dispUnc展示最终测量值的计算过程时，需要通过如下设置，来设定需要生成计算表达式：

In [11]:
from analyticlab.uncertainty.unc import Uncertainty
Uncertainty.process = True

### 3.2 通过数学关系式得到Uncertainty

通过数学运算符`+`、`-`、`*`、`/`、`//`、`**`<i>（“/”与“//”的作用于LSym类似，即除号的形式不同）</i>，或者`amath`库中的函数，给出各个Measure之间，或者Measure与Uncertainty之间，Uncertainty与Uncertainty之间的数学关系式，从而得到Uncertainty：

In [12]:
#先创建Measure
a = Measure('15.03 14.89 14.94 15.11 14.98', ins.米尺_厘米_2, sym='a')
b = Measure('7.61 7.70 7.55 7.63 7.66', ins.米尺_厘米_2, sym='b')
c = Measure('5.73 5.75 5.76 5.75 5.77', ins.米尺_厘米_2, sym='c')
#然后通过数学关系式的得到Uncertainty
V = a*b*c

<i>注意：由于三角函数不是线性函数，因此不能三角函数不能用于Measure、Uncertainty的计算。如果确实需要，应先把三角函数展开成泰勒多项式，再计算多项式的不确定度。</i>

#### 3.2.1 简化表示不确定度的数学关系式

对于较复杂的数学关系式，新增一个系数可能就会使不确定度的计算式变得繁琐很多。因此，如果最终测量值的数学表达式的最外层为乘除式，且最外层有系数，如$\cfrac{\pi}{4}(d_1^2-d_1^2)h$、$y=\cfrac{8Fl}{3m}$，可以将最外层系数去掉，换成$(d_1^2-d_0^2)h$、$y=\cfrac{Fl}{m}$，这样既不会影响不确定度结果，又能大幅简化不确定度计算式。

### 3.3 获得合成不确定度

注意Uncertainty和Measure获得合成不确定度时的方法是不同的，在Uncertainty中，通过调用`result()`方法，可以获得合成不确定度数值：

In [13]:
V.result()

5.4

### 3.4 获得扩展不确定度

默认情况下，`result()`方法返回的是P=0.6826时的合成不确定度。如果希望获得的是最终测量值的扩展不确定度，则需要调用**最终测量值的`setK(K)`方法**，给出K值。给定K值后，再调用其`result()`方法，得到扩展不确定度。允许的K值请见2.4中的列表。

## 4.dispUnc函数：展示不确定度计算全过程

### 4.1 函数说明

dispUnc函数可以根据**各个独立测量**和**最终测量结果的数学关系式**，生成完整的不确定度计算过程。dispUnc函数如下：

`def dispUnc(measures, resUnc, resValue, resSym, resUnit, resDescription=None)`

#### 相关参数说明：
* `measures`：构成不确定度的全部测量（Measure）组成的列表。
* `resUnc`：最终测量结果的不确定度，可以是Uncertainty或Measure类型。
* `resValue`：最终测量结果的数值，Num类型。
* `resSym`和`resUnit`：最终测量结果的符号和单位，以字符串形式给出。
* `resDescription`：对最终测量结果的描述，比如“故空心圆柱体的体积为”。可以不给出。给出时，会在最终测量结果开始时，对其进行语言描述，如“故空心圆柱体的体积为$V=(1.7752 \pm 0.0512){\rm cm^3} \qquad {\rm P}=0.6826$”

### 4.2 应用举例

下面以空心圆柱体体积不确定度计算为例，来演示如何通过`dispUnc`函数得到体积的**扩展不确定度**（K=2）及其计算过程：

In [14]:
from analyticlab import const, latexoutput as lp
from analyticlab.numitem import NumItem
from analyticlab.uncertainty import ins
from analyticlab.uncertainty.measure import Measure
#以NumItem数组定义数据
d1 = NumItem('15.03 14.89 14.94 15.11 14.98', sym='d_1')
d0 = NumItem('7.61 7.70 7.55 7.63 7.66', sym='d_0')
h = NumItem('5.73 5.75 5.76 5.75 5.77', sym='h')
#由数组间的数学关系式得到体积V
V = const.PI/4 * (d1**2-d0**2) * h
#取体积V的均值
V = V.mean()
#定义测量
d1 = Measure(d1, ins.米尺_厘米_2, description='外径$d_1$')
d0 = Measure(d0, ins.米尺_厘米_2, description='内径$d_0$')
h = Measure(h, ins.米尺_厘米_2, description='高度$h$')
#由测量间的数学关系式得到体积不确定度uV（这里是用简化后的数学关系式）
uV = (d1**2-d0**2) * h
#设置扩展不确定度K值
uV.setK(2)
#使用dispUnc函数生成并展示不确定度
lp.dispUnc([d1,d0,h], uV, V, 'V', 'cm^3', '故空心圆柱体的体积为').show()

<IPython.core.display.Math object>

## \*5.std函数模块：计算标准偏差数值

### \*5.1 导入std模块

通过如下指令实现analyticlab.uncertainty.std模块的导入：

In [15]:
from analyticlab.uncertainty import std

### \*5.2 函数列表

**std模块支持3种标准偏差计算方法，模块函数如下：**
* `def Bessel(item, remainOneMoreDigit=False)` - 贝塞尔公式法
* `def Range(item, remainOneMoreDigit=False)` - 极差法
* `def CollegePhysics(item, remainOneMoreDigit=False)` - 大学物理实验的标准偏差计算

其中参数`item`为样本的NumItem数组；`remainOneMoreDigit`用于控制得到的结果是否多保留一位有效数字。返回值为Num类型的标准偏差数值。

## \*6.ACategory函数模块：计算A类不确定度

### \*6.1 导入ACategory模块

通过如下指令实现analyticlab.uncertainty.ACategory模块的导入：

In [16]:
from analyticlab.uncertainty import ACategory

### \*6.2 函数列表

**ACategory模块支持4种A类不确定度计算方法，模块函数如下：**

* 单个样本的A类不确定度计算：
    * `def Bessel(item, process=False, needValue=False, remainOneMoreDigit=False)` - 贝塞尔公式法
    * `def Range(item, process=False, needValue=False, remainOneMoreDigit=False)` - 极差法
    * `def CollegePhysics(item, process=False, needValue=False, remainOneMoreDigit=False)` - 大学物理实验的A类不确定度计算
* 多个样本的组合A类不确定度计算：
    * `def CombSamples(items, method='auto', process=False, needValue=False, sym=None, unit=None, remainOneMoreDigit=False)` - 组合不确定度

### \*6.3 单个样本的A类不确定度计算

在`Bessel`、`Range`、`CollegePhysics`函数中，参数`item`为单个样本的NumItem数组；`remainOneMoreDigit`用于控制得到的结果是否多保留一位有效数字。默认情况下，返回值为A类不确定度的Num数值。

`process`、`needValue`用于控制是否生成A类不确定度的计算过程，以及在生成计算过程的同时是否返回A类不确定度数值。

### \*6.4 多个样本的组合A类不确定度计算

在`CombSamples`函数中，参数`items`为多个样本的NumItem数组组成的list；`method`为使用何种方法计算每个样本的A类不确定度，默认为`'auto'`，其可取的值请见2.2；`remainOneMoreDigit`用于控制得到的结果是否多保留一位有效数字。默认情况下，返回值为组合A类不确定度的Num数值。

参数`process`、`needValue`用于控制是否生成合并A类不确定度的计算过程，以及在生成计算过程的同时是否返回合并A类不确定度数值。当`process=True`时，多个样本整体对应的符号`sym`必须给出，单位`unit`可自行选择是否选择是否给出。不生成计算过程时，参数`sym`、`unit`都不需要给出。

## \*7.BCategory函数模块：计算B类不确定度

### \*7.1 导入BCategory模块

通过如下指令实现analyticlab.uncertainty.BCategory模块的导入：

In [17]:
from analyticlab.uncertainty import BCategory

### \*7.2 B类不确定度计算

`b`函数用于计算B类不确定度，该函数如下：

`def b(instrument, sym=None, process=False, needValue=False, remainOneMoreDigit=True)`

其中参数`instrument`为测量仪器（Ins类型）；`remainOneMoreDigit`用于控制得到的结果是否多保留一位有效数字。默认情况下，返回值为B类不确定度的Num数值。

参数`process`、`needValue`用于控制是否生成B类不确定度的计算过程，以及在生成计算过程的同时是否返回B类不确定度数值。当`process=True`时，测量的符号`sym`必须给出，否则不需要给出。